# Privacy Preserving Machine Learning Practical Session 1

Practical Session adapted from the Course of Olivier Cappé 

Course page: https://www.di.ens.fr/olivier.cappe/Courses/IASD-DP/

<div style="text-align: right"> 
Sorbonne Université, MS2A, Master 2
</div>

## Goal of the practical session 

The objective of this practical session is to introduce methods often regarded as privacy-preserving within parts of the data processing community. While these approaches are widely used as "best efforts" for privacy, they come with significant failure modes that must be understood. Exploring these concepts and their vulnerabilities is essential before advancing to more robust privacy solutions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pylab as pl
import matplotlib.patches as patches

## Table of contents
1. [Part 1: K-Anonymity + L-Diversity + T-Closeness](#part1)
    - [Presenting K-anonymity](#part1sec1)
        - [Question 1 (span and split functions)](#part1sec2)
        - [Question 2 (Implementing the Mondrian partitionning algorithm)](#part1sec3)
        - [Question 3 (Analyzing the partition)](#part1sec4)
        - [Question 4 (Limitations of k-anonimity)](#part1sec5)
    - [Presenting L-diversity (naive way)](#part1sec6)
        - [Question 5 (Implementing l-diversity)](#part1sec7)
        - [Question 6 (Limitations of l-diversity)](#part1sec8)
    - [Presenting T-closeness](#part1sec9)
        - [Question 7 (Implementing t-closeness)](#part1sec10)
        - [Question 8 (Analysis of the usefulness of t-closeness)](#part1sec11)
1. [Part 2: Brittlness of privacy through aggregation (a.k.a. attacking ML models)](#part2)
    - [Question 1 (Warm-Up: guessing the target vetcore from aggregated observations)](#part2sec1)
    - [Question 2 (Warm-Up: verifying the attack)](#part2sec2)
    - [Question 3 (Model inversion attack on a logstic regression)](#part2sec3)
    - [Question 4 (Success with respect to side information)](#part2sec4)

# Part 1: K-anonymity, L-diversity, T-closeness and their limitations <a id="part1"></a>

In the late 1990s Sweeney published several articles in which she proposed the concept of k-anonymity. Among them we cite the most famous one [1] published in 2002. At the time when anonymization was referring to the concept now known as pseudonymization. Pseudonymization is the replacement of all directly identifying data (such as the social security number) with a random value (pseudonym). As we mentioned during the first lecture, Sweeney showed in [1] that it was possible to re-identify a pseudonymized relational database (SQL or more generally a tabular file) and proposed a simple notion called "k-anonimity" to prevent these kind of leak from hapenning.

## Presenting K-anonymity <a id="part1sec"></a>

Suppose we have a dataset $D$ containing $n$ entries. Each entry in $D$ consists of a set of attributes—such as age, gender, and zip code—that are non-sensitive on their own but can act as quasi-identifiers. When combined, these attributes form a "super-identifier" that may uniquely identify an individual, even in large datasets (e.g., the combination of gender, age, and zip code might be unique to a single person; see [1] for further details). Additionally, let us assume the dataset includes a single sensitive attribute (such as income) that we aim to protect. While $k$-anonimity can be generalized to datasets with multiple sensitive attributes or those lacking a clear distinction between quasi-identifiers and sensitive attributes, we will focus on this simplified case for this practical session.

The concept of $k$-Anonymity protects individual privacy by grouping records into clusters of at least $k$ individuals. In essence, a dataset satisfies $k$-anonymity if every entry shares its quasi-identifiers with at least $k-1$ other entries. To achieve this, the dataset is partitioned into groups of at least $k$ records, and the quasi-identifiers within each group are replaced with aggregated values. This aims to ensure that an adversary, even with full knowledge of a person’s quasi-identifiers, can only infer the group to which the person might belong and not whether the person is actually in the dataset.

**Implementing k-anonimity:** Turning a dataset into a $k$-anonymous dataset is a complex problem. Meyerson and Williams have shown in [2] that finding the optimal $k$-anonymous transformation of the database is NP-difficult. Fortunately, several practical algorithms exists that often produce "good enough" results by employing greedy search techniques. In this notebook, we will explore the so-called "Mondrian" (see [3] for more details) algorithm, which uses a greedy search method to partition the original data into smaller and smaller groups. The algorithm assumes that we have converted all attributes into numerical or categorical values and that we are able to measure the "span" of a given data attribute ("span" will be detailed later). The algorithm proceeds then as follows to partition the data into $k$ groups:

1. Initialize the final set of partitions to an empty set $P_{final} = \{\}$.
2. Initialize the temporary set of paritions to a set containing a partition with the entire dataset $P_{temp} = \{\{1, 2,\dots ,n\}\}$.
4. While there are partitions in the temporary set, pop out one partition from it.
  * Compute the relative spans of all columns in the partition.
  * Sort the resulting columns by their span (in descending order) and iterate over them. For each column,
      * Try to split the partition along that column using the median of the column values as the split point.
      * Check if the resulting partitions are valid according to k-anonymity (and possibly additional) criteria.
      * If yes, add the two new partitions to the temporary set and break out of the loop.
  * If no column produced a valid split, add the original partition to the set of final partitions.
5. Return the final set of partitions

After obtaining the partitions we still need to aggregate the values of the quasi identifiers and the sensitive attributes in each $k$-anonymous group. For this, we can e.g. replace numerical attributes with their range (e.g. "age: 24-28") and categorical attributes with their union (e.g. "employment-group: [self-employed, employee, worker]"), though other aggregations are possible. An open source tool developed by the Technological University of Munich, ARX (see [4]) allows anonymisation according to many models from original data in tabular format. The tool also makes it possible to estimate the risks of de-anonymization according to the models presented above, and even in a finer way by calculating the distribution of the probabilities of de-anonymization and not simply the maximum values.

- [1] [k-Anonymity: A Model For Protecting Privacy](https://epic.org/privacy/reidentification/Sweeney_Article.pdf)
- [2] [On the Complexity of Optimal K-Anonymity](http://www.aladdin.cs.cmu.edu/papers/pdfs/y2004/kanonim.pdf)
- [3] [Mondrian - Multidimensional k-Anonymity](https://www.utdallas.edu/~muratk/courses/privacy08f_files/MultiDim.pdf)
- [4] [Putting statistical disclosure control into practice: The ARX data anonymization tool](https://link.springer.com/chapter/10.1007/978-3-319-23633-9_6)



## Dataset

We will be working with a dataset from the US Census (also known as the Adult dataset). You can read about the dataset [here](https://archive.ics.uci.edu/ml/datasets/census+income). The following line loads the dataset from a text file in `pandas DataFrame` format: this keeps the attributes in their original form and will be more convenient to work with. If you prefer working with a numpy array, set `as_frame=False` but it is not recommended in general.

In [ ]:
names = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
)

categorical = set((
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'sex',
    'native-country',
    'race',
    'income',
))
df = pd.read_csv("adult.all.txt", sep=", ", header=None, names=names, index_col=False, engine='python');
df.head()

### Question 1 (span and split functions) <a id="part1sec2"></a>

Implement a function called ``get_spans`` that computes the spans of all columns for a given partition of a dataframe. For numerical columns, the span is defined as the difference between the maximum and minimum values of the partition, while for categorical columns, it is the count of unique values. Additionally, implement a ``split`` function that divides a dataframe partition into two subsets based on a specified numerical column. The function should return two ensembles: one containing all rows with values below the median of the specified column, and the other containing rows with values equal to or above the median.

In [ ]:
def get_spans(df, partition, scale=None):
    """
    Parameters
    ----------
    df: 
        The dataframe of reference
    partition: 
        The partition for which to calculate the spans
    scale: 
        If given, the spans of each column will be divided by the value in `scale` for that column
    Returns
    -------                 
    spans: 
        The spans of all columns in the partition
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

In [ ]:
def split(df, partition, column):
    """
    Parameters
    ----------
    df: 
        The dataframe of reference
    partition: 
        The partition to split
    column: 
        The column along which to split
    
    Returns
    ------- 
    splits: 
        A tuple containing a split of the original partition
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

### Question 2 (Implementing the Mondrian partitionning algorithm) <a id="part1sec3"></a>

Now that all helper functions are ready, we can proceed to implement the Mondrian algorithm as described. Your task is to develop the partitioning algorithm using a $k$-anonymous criterion for the generated partitions. To achieve this, start by completing the ``is_k_anonymous`` function, which checks whether a newly created partition meets the $k$-anonymity requirement. Once this function is ready, integrate it into the ``partition_dataset`` function, which will generate a $k$-anonymous partition proposal for the given dataset.

In [ ]:
def is_k_anonymous(df, partition, sensitive_column, k=3):
    """
    Parameters
    ----------
    df: 
        The dataframe on which to check the partition.
    partition: 
        The partition of the dataframe to check.
    sensitive_column: 
        The name of the sensitive column
    k: 
        The desired k
    
    Returns
    ------- 
    verif: 
        True if the partition is valid according to our k-anonymity criteria, False otherwise.
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

def partition_dataset(df, feature_columns, sensitive_column, scale, is_valid):
    """
    Parameters
    ----------
    df: 
        The dataframe to be partitioned.
    feature_columns: 
        A list of column names along which to partition the dataset.
    sensitive_column: 
        The name of the sensitive column (to be passed on to the `is_valid` function)
    scale: 
        The column spans as generated before.
    is_valid: 
        A function that takes a dataframe and a partition and returns True if the partition is valid.
    
    Returns
    ------- 
    valid_part: 
        A list of valid partitions that cover the entire dataframe.            
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

### Question 3 (Analyzing the partition) <a id="part1sec4"></a>

Let's apply this to our dataset. To keep things manageable, we will focus on just two columns for partitioning: 'age' and 'hours of work per week', which will serve as our pseudo-identifiers. Furthermore, we will treat 'income' as the sensitive attribute. After running the partitioning algorithm, we visualize the resulting partitions to better understand how the data is divided.
To do this, we run a functions to determine the rectangular bounds of each partition along the two selected columns. By plotting these rectangles, we can see how the partitioning function segments the dataset. If we restrict the partitioning to only these two columns, the resulting rectangles should be non-overlapping and collectively cover the entire dataset. Run the code below and analyze the result we get. What can you say about the way the dataset is partitionned ?

In [ ]:
# Create the partition
feature_columns = ['age', 'hours-per-week']
sensitive_column = 'income'
final_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, is_k_anonymous)

In [ ]:
def build_indexes(df):
    indexes = {}
    for column in categorical:
        values = sorted(df[column].unique())
        indexes[column] = { x : y for x, y in zip(values, range(len(values)))}
    return indexes

def get_coords(df, column, partition, indexes, offset=0.1):
    if column in categorical:
        sv = df[column][partition].sort_values()
        l, r = indexes[column][sv[sv.index[0]]], indexes[column][sv[sv.index[-1]]]+1.0
    else:
        sv = df[column][partition].sort_values()
        next_value = sv[sv.index[-1]]
        larger_values = df[df[column] > next_value][column]
        if len(larger_values) > 0:
            next_value = larger_values.min()
        l = sv[sv.index[0]]
        r = next_value
    # we add some offset to make the partitions more easily visible
    l -= offset
    r += offset
    return l, r

def get_partition_rects(df, partitions, column_x, column_y, indexes, offsets=[0.1, 0.1]):
    rects = []
    for partition in partitions:
        xl, xr = get_coords(df, column_x, partition, indexes, offset=offsets[0])
        yl, yr = get_coords(df, column_y, partition, indexes, offset=offsets[1])
        rects.append(((xl, yl),(xr, yr)))
    return rects

def get_bounds(df, column, indexes, offset=1.0):
    if column in categorical:
        return 0-offset, len(indexes[column])+offset
    return df[column].min()-offset, df[column].max()+offset

In [ ]:
# Compute the bounding rects of all partitions that we created
indexes = build_indexes(df)
column_x, column_y = feature_columns[:2]
rects = get_partition_rects(df, final_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [ ]:
# Plot the rectangles
def plot_rects(df, ax, rects, column_x, column_y, edgecolor='black', facecolor='none'):
    for (xl, yl),(xr, yr) in rects:
        ax.add_patch(patches.Rectangle((xl,yl),xr-xl,yr-yl,linewidth=1,edgecolor=edgecolor,facecolor=facecolor, alpha=0.5))
    ax.set_xlim(*get_bounds(df, column_x, indexes))
    ax.set_ylim(*get_bounds(df, column_y, indexes))
    ax.set_xlabel(column_x)
    ax.set_ylabel(column_y)

pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, rects, column_x, column_y, facecolor='r')

### Question 4 (Limitations of k-anonimity) <a id="part1sec5"></a>

Beyond the observed limitation above, what are the key limitations of this approach? Explain why the following scenarios stand out as critical issues for $k$-anonimity

- All $k$ records in a group share the same sensitive attribute? 
- An adversary knows $k-1$ individuals in a group with identical quasi-identifiers?
 



## Presenting L-diversity (naive way) <a id="part1sec6"></a>

The primary strength of $k$-anonymity lies in its simplicity and intuitive nature (a valuable asset when explaining privacy guarantees to non-experts). However, as we have just seen, the model has critical limitations that undermine its effectiveness in real-world scenarios. On its own, $k$-anonymity fails to deliver robust anonymity guarantees. To address these shortcomings, the model is extended to $l$-diversity, which we will explore next. l-diversity, first introduced in 2007 in [5], ensures that each k-anonymous group contains at least $l$ different values of the sensitive attribute. Therefore, even if an adversary can identify the group of a person they still would not be able to find out the value of that person's sensitive attribute with certainty. To implement l-diversity, we can do the following:

- Modify our `is_valid` function to not only check for the size of the partition but also ensure that the values of the sensitive attribute are diverse enough.
- Modify the `split` function to produce splits that are diverse (if possible)

**Remark:** Here we will only implement the first point to keep things simple, please keep in mind that this is not the smartest way to implement $l$-diversity, as our "naive" splitting function might produce invalid splits even when it would actually be possible to produce a valid one.

- [5] [l-Diversity: Privacy Beyond k-Anonymity](https://personal.utdallas.edu/~muratk/courses/privacy08f_files/ldiversity.pdf)


### Question 5 (Implementing l-diversity) <a id="part1sec7"></a>

Implement a ``is_l_diverse`` function that returns `True` if a given partition contains at least `l` different values of the sensitive attribute, `False` otherwise. Then using the ``partition_dataset`` function, compute a partition that satisfy $l$-diversity for $l=2$  

In [ ]:
def is_l_diverse(df, partition, sensitive_column, l=2):
    """
    Parameters
    ----------
    df: 
        The dataframe for which to check l-diversity
    partition: 
        The partition of the dataframe on which to check l-diversity
    sensitive_column: 
        The name of the sensitive column
    l: 
        The minimum required diversity of sensitive attribute values in the partition
        
    Returns
    ------- 
    verif: 
        True if the partition is valid according to our l-diversity criteria, False otherwise.
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

In [ ]:
final_l_diverse_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, [TO COMPLETE])

In [ ]:
#Plot the partition
column_x, column_y = feature_columns[:2]
l_diverse_rects = get_partition_rects(df, final_l_diverse_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, l_diverse_rects, column_x, column_y, edgecolor='b', facecolor='b')

### Question 6 (Limitations of l-diversity) <a id="part1sec8"></a>

What are the key limitations of this approach? Can you think of a scenario that would stand as critical for $l$-diversity? 

**Hint** Even when using l-diversity an adversary could still learn some information about a person's sensitive attribute using probabilistic reasoning or prior knowledge. 

## Presenting T-closeness <a id="part1sec9"></a>

t-closeness, also introduced in 2007 in [3], demands that the statistical distribution of the sensitive attribute values in each k-anonymous group is "close" to the overall distribution of that attribute in the entire dataset. Typically, assuming the sensitive attribute has $m$ modalities, the closeness between two probability vectors $p=(p_1,...,p_m)$ and $q=(q_1,...,q_m)$ over these modalities can be measured using e.g. the Kullback-Leibler (KL) divergence defined as follows: $$D_{KL}(p,q) = \sum_{i=1}^{m}p_i \log(\frac{p_i}{q_i}).$$

If we ensure that between any two paritions the divergence between the sensitive attrivutes within the groups is bounded by some small constant $t$, the adversary could then only learn a limited amount of information from comparing the distribution of the values in the group to the distribution in the entire dataset.


### Question 7 (Implementing t-closeness) <a id="part1sec10"></a>

Implement a version of the `is_valid` function that returns `True` if the partition is diverse enough and `False` otherwise. To measure diversity, compute the total variation distance (easier to implement compared to KL) between the empirical probability distribution of the sensitive attribute over the entire dataset vs. the distribution over the partition (the total variation distance is the maximum pointwise absolute difference between the two distributions). 

In [ ]:
# Here we generate the global frequencies for the sensitive column 
global_freqs = {}
total_count = float(len(df))
group_counts = df.groupby(sensitive_column)[sensitive_column].agg('count')
for value, count in group_counts.to_dict().items():
    p = count/total_count
    global_freqs[value] = p

In [ ]:
def is_t_close(df, partition, sensitive_column, global_freqs, t=0.1):
    """
    Parameters
    ----------
    df: 
        The dataframe for which to check t-closeness
    partition: 
        The partition of the dataframe on which to check t-closeness
    sensitive_column: 
        The name of the sensitive column
    global_freqs: 
        The global frequencies of the sensitive attribute values
    t: 
        The maximum allowed distance
        
    Returns
    ------- 
    verif: 
        True if the partition is valid according to our t-closeness criteria, False otherwise.
    """
    # TO COMPLETE
    pass # this is just to avoid error while the function is empty

In [ ]:
final_t_close_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, lambda *args: is_k_anonymous(*args) and is_t_close(*args, global_freqs))

column_x, column_y = feature_columns[:2]
t_close_rects = get_partition_rects(df, final_t_close_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])
pl.figure(figsize=(20,20))
ax = pl.subplot(111)
plot_rects(df, ax, t_close_rects, column_x, column_y, edgecolor='b', facecolor='b')

### Question 8 (Analysis of the usefulness of t-closeness) <a id="part1sec11"></a>

Below, we run a partition procedure for $t$-closeness and plot the results. What do you think about the usefulness of this partition ?

# Part 2: Brittleness of privacy through aggregation (a.k.a. attacking ML models) <a id="part2"></a>

Beyond standard privacy definition (such as $k$-anonymity, $l$-diversity, and $t$-closeness, each of which has well-documented limitations), there’s a common but misplaced belief that data integrated into aggregate statistical queries (e.g., during the training of large machine learning models) inherently benefits from a "hide in the crowd" effect. Unfortunately, this assumption does not hold in general, as we will demonstrate in the second part of this session. For simplicity, we focus on straightforward cases where "playing the role of the adversary" can be executed in just a few minutes. However, it is important to note that, with more time and effort, similar attacks can be extended to extract sensitive information from the training data of even highly complex models (such as large language models) as we discussed in class.


### Another Dataset 

We will be working with the Pima Indians Diabetes Dataset, a well-known dataset originally collected by the National Institute of Diabetes and Digestive and Kidney Diseases. You can read more about the dataset [here](https://www.openml.org/search?type=data&status=active&id=37&sort=runs). The following line loads the dataset into a pandas DataFrame, preserving the original structure of the attributes. 


In [ ]:
# Download data from https://openml.org/search?type=data&status=active&id=37
pima = fetch_openml(name='diabetes', version=1, parser='auto')
# Convert target to numeric +/- 1
outcome = np.where(pima.target == 'tested_positive', 1, -1)
n = outcome.size
# Standardize the data and add intercept

data = np.concatenate((np.ones((n, 1)), StandardScaler().fit_transform(pima.data)), axis=1)

### Question 1 (Warm-Up: guessing the target vetcore from aggregated observations) <a id="part2sec1"></a>

In this first exercise, we assume that we are an adversary having access to a set of random vectors $z_1, ..., z_K$ that represent the dot product between the label (target) vector $y \in \{-1,1\}^n$ and a set of random vector $w_1, ..., w_K \in \mathbb{R}^n$. Assume we the adversary knows the matrix $W=[w_1, ..., w_K]$, how could they guess the value of the target $y$ for the dataset at hand? Can you also guess how the value of $K$ would would influence the quality of your strategy ? 

### Question 2 (Warm-Up: verifying the attack) <a id="part2sec2"></a>

- Complete the code below with your idea of how to predict the labels $y$ from the observed random dot products $z$
- Implement and observe the ROC (Receiver Operating Characteristic) and DET (Detection Error Tradeoff) curves associated with your guess.
- By varying $K$, find the minimal value of $K$ that give non trivial recovery performance. For what value of $K$ can you recover 50% of the labels at 5% FPR? 
- What changes if we now assume that some of the labels in $y$ are known?

In [ ]:
# Use label mapped to {-1,1}
y = outcome
n = y.size
# Observe K dot products with random sign vectors
K = 50
# Do it several times to get statistically significant results
nMC = 50    # Number of Monte Carlo runs
estim = np.zeros((nMC, n))
for iMC in range (nMC):
    # Observe dot product with random signs matrix
    W = np.power(-1, np.random.binomial(1, 0.5, size=(K,n)))
    z = np.matmul(W, y)

    # Begin TODO: Use a better estimation method!
    estim[iMC, :] = np.random.normal(size=(1,n))
    # End TODO

### Question 3 (Model inversion attack on a logistic regression) <a id="part2sec3"></a>

In this part, we provide a utility function that can be used to compare two ROC curves on the same plot and examine the result of using logistic regression on this dataset (you can experiment with different train and test splits). 

In [ ]:
def display_two_roc_curves_from_predictions(y_true1, y_score1, label1, y_true2, y_score2, label2):
    """
    Plots two ROC curves from given true labels and predicted scores.

    Parameters:
    ----------
    y_true1 (array-like): 
        True binary labels for the first set of predictions.
    y_score1 (array-like): 
        Target scores for the first set of predictions.
    label1 (str): 
        Label for the first ROC curve.
    y_true2 (array-like): 
        True binary labels for the second set of predictions.
    y_score2 (array-like): 
        Target scores for the second set of predictions.
    label2 (str): 
        Label for the second ROC curve.
    """
    fpr1, tpr1, tresh = metrics.roc_curve(y_true1, y_score1)
    auc1 = metrics.roc_auc_score(y_true1, y_score1)
    plt.plot(fpr1, tpr1, label=label1+' (AUC={0:.2f})'.format(auc1))
    fpr2, tpr2, tresh = metrics.roc_curve(y_true2, y_score2)
    auc2 = metrics.roc_auc_score(y_true2, y_score2)
    plt.plot(fpr2, tpr2, label=label2+' (AUC={0:.2f})'.format(auc2))
    plt.legend(loc=0)
    plt.grid(True)
    plt.xlabel('FPR')
    plt.ylabel('TPR')

In [ ]:
# Fit logistic regression to the data (without intercept as we explicitly added it to the data already)
X = data
y = outcome
(n, d) = X.shape
n_train = 200
n_test = n - n_train
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n_train, random_state=45)
logreg = LogisticRegression(fit_intercept=False)
logreg.fit(X_train, y_train)
# Plot training and test performance as ROCs
y_train_pred = logreg.predict_proba(X_train)[:,1]
y_test_pred = logreg.predict_proba(X_test)[:,1]
print('Training and test accuracies: {0:.2f}, {1:.2f}'.format(np.sum(y_train*(y_train_pred-0.5) > 0)/n_train,
    np.sum(y_test*(y_test_pred-0.5) > 0)/n_test))
display_two_roc_curves_from_predictions(y_train, y_train_pred, 'Training performance',
    y_test, y_test_pred, 'Test performance')
plt.show()

Implement a function to compute the gradients of the logistic loss for each training example in a dataset. These gradients will serve as the foundation for constructing a statistical test designed to infer the value of an unknown label.  The scenario assumes that you are and adversary having access to $K$ known labels, along with their feature matrix and a pre-trained logistic regression model. By analyzing the gradients, you can derive statisticl insights about the unknown label, effectively "guessing" its value based on the model’s behavior and the known data points. This approach highlights how gradient information can be exploited to infer sensitive information, even when only partial labels are available.

In [ ]:
def logreg_gradients(X, y, logreg):
     """
     Compute the gradients of the logistic regression loss function with respect to model parameters.

     Parameters:
     ----------
     X (numpy.ndarray): 
         The input feature matrix of shape (n, d), where n is the number of samples and d is the number of features.
     y (numpy.ndarray): 
         The target labels of shape (n,), where n is the number of samples.
     logreg (object): 
         A trained logistic regression model.

     Returns:
     ----------
     numpy.ndarray: 
         The gradient matrix of shape (n, d), where each row corresponds to the gradient for a single sample.
     """
     # TO COMPLETE
     pass # this is just to avoid error while the function is empty

### Question 4 (Success with respect to side information) <a id="part2sec4"></a>

Experimenting with random subsets of the data (using the provided code as example), for what values of $K$ does this approach start to give non trivial recovery performance? Try different values of the training size $n$; how does it modify the previous conclusion? What aspect of the model appears to be important for ensuring the success of the recovery? 

In [ ]:
# We will work on recovering data from the training sample
X = X_train
y = y_train
(n, d) = X.shape
# Compute the probabilistic predictions from the model
p_hat = logreg.predict_proba(X)[:,1]
# Try reconstruction attack assuming K known labels on randomly chosen data subsets
K = 50 # Number of known labels
# Number of Monte Carlo experiments
nMC = 500
y_true = np.zeros(nMC)
y_guess = np.zeros(nMC)
y_pred = np.zeros(nMC)


for i in range(nMC):
    # Draw a random permutation and try to reconstruct y[ind[0]] assuming knowledge of y[ind[1:p+1]]
    ind = np.random.permutation(n)
    # True label
    y_true[i] = y[ind[0]]
    # Model prediction
    y_pred[i] = p_hat[ind[0]]
    # Begin TODO: Your guess here
    y_guess[i] = np.random.normal(size=1)
    # End TODO


display_two_roc_curves_from_predictions(y_true, y_pred, 'Model prediction',
    y_true, y_guess, 'Attack reconstruction')
plt.title("n={}, K={}".format(n, K))
plt.show()